In [ ]:
from PyFoam.Applications.CloneCase import CloneCase
from PyFoam.Applications.ClearCase import ClearCase
from PyFoam.Applications.CreateBoundaryPatches import CreateBoundaryPatches
from PyFoam.Applications.ChangeBoundaryName import ChangeBoundaryName
from PyFoam.Applications.WriteDictionary import WriteDictionary
from PyFoam.Applications.Runner import Runner
import shutil, os
# Import vtk module
from vtk import *

In [ ]:
# Assume that we already sourced .bashrc with OpenFOAM-5.0
#openfoam_source = os.path.abspath("/opt/openfoam5/etc/bashrc")
#os.system("source "+openfoam_source)
# Clone case from the existing one
srcpath = os.path.join(os.environ['FOAM_TUTORIALS'],"incompressible/icoFoam/cavity/cavity")
CloneCase(args=["--no-vcs",srcpath,"cavity","--force","--no-pyfoam"])

In [ ]:
Runner(args=["--silent","--no-server-process","blockMesh","-case","cavity"])

In [ ]:
#Change BC name just to test functionality
ChangeBoundaryName(args=["cavity","movingWall","slideWall"])
#CreateBoundaryPatches(args=["--overwrite","--filter=fixedWalls","--default={'type':'fixedValue','value':'uniform (0 0 0)'}","./cavity/0/U"])
#CreateBoundaryPatches(args=["--overwrite","--filter=movingWall","--default={'type':'fixedValue','value':'uniform (1 0 0)'}","./cavity/0/U"])

In [ ]:
CreateBoundaryPatches(args=["--clear-unused","--overwrite","./cavity/0/U"])
CreateBoundaryPatches(args=["--clear-unused","--overwrite","./cavity/0/p"])

In [ ]:
CreateBoundaryPatches(args=["--overwrite","--filter=fixedWalls","--default={'type':'fixedValue','value':'uniform (0 0 0)'}","./cavity/0/U"])
CreateBoundaryPatches(args=["--overwrite","--filter=slideWall","--default={'type':'fixedValue','value':'uniform (2 0 0)'}","./cavity/0/U"])

In [ ]:
WriteDictionary(args=["./cavity/system/controlDict","startTime","0"])
WriteDictionary(args=["./cavity/system/controlDict","endTime","1"])
WriteDictionary(args=["./cavity/system/controlDict","deltaT","0.005"])
WriteDictionary(args=["./cavity/system/controlDict","writeControl","runTime"])
WriteDictionary(args=["./cavity/system/controlDict","writeInterval","0.1"])

In [ ]:
# clear previous calculation results and run application
Runner(args=["--complete-clear","--pyfoam-stuff-clear","--silent","--no-server-process","icoFoam","-case","cavity"])

In [ ]:
# convert OpenFOAM results into legacy VTK format
os.system("foamToVTK -case ./cavity -latestTime -ascii")

In [ ]:
#################################33
# Define path to VTK results file
filename=os.path.join(os.getcwd(),"cavity/VTK/cavity_200.vtk")

# Read the source file.
reader = vtkUnstructuredGridReader()
reader.SetFileName(filename)
reader.Update()
# Define output port (whatever)
reader_out =  reader.GetOutputPort()

In [ ]:
# Define mapper for mesh
meshMapper = vtkDataSetMapper()
meshMapper.SetInputConnection(reader_out)

In [ ]:
# convert to CellData (because OpenFOAM writes VTK results as Field dataset attribute)
# Field represent an array of data arrays, see VTK User's Guide, "VTK File Formats"
# We need to convert it to cell or point Data
# First do for cells
cellData = vtkFieldDataToAttributeDataFilter()
cellData.SetInputConnection(reader.GetOutputPort())
cellData.SetInputFieldToCellDataField()
cellData.SetOutputAttributeDataToCellData()
# Choose whether to normalize results or not
#cellData.DefaultNormalizeOn()

# Define which scalar to show (velocity is also a scalar with 3 components in array)
"""
Hint: to define a scalar one should give 3 arguments (at least)
- component number, type int - what is it I don't know
- array name (the name of scalar array which is the name of a scalar to show), type str
- component in scalar (e.g. in array 'p' there is 1 component; in array 'U' there is 3 components)
"""
#cellData.SetScalarComponent(0, 'U', 0)
##########################################

In [ ]:
# Second, do for points
pointData = vtkFieldDataToAttributeDataFilter()
pointData.SetInputConnection(reader.GetOutputPort())
pointData.SetInputFieldToPointDataField()
pointData.SetOutputAttributeDataToPointData()
pointData.SetScalarComponent(0, 'U', 0)
#pointData.SetVectorComponent(0,'U',1)

In [ ]:
# Define a mapper for the data
res_mapper = vtkDataSetMapper()
# Define a range to show
res_mapper.SetScalarRange(-0.5,2)
##################################

# Define input connection - cells or points
#res_mapper.SetInputConnection(cellData.GetOutputPort())

res_mapper.SetInputConnection(pointData.GetOutputPort())

In [ ]:
# Define actor for results mapper
resActor=vtk.vtkActor()
resActor.SetMapper(res_mapper)
# Define actor for mesh
meshActor=vtk.vtkActor()
# Mesh color in RGB forma: (0,0,0) - black
meshActor.GetProperty().SetColor(0.0,0.0,0.0)
meshActor.GetProperty().SetLineWidth(2)
# Define mesh opacity
meshActor.GetProperty().SetOpacity(0.5)
meshActor.SetMapper(meshMapper)
# Define show regime - Wireframe to draw mesh lines and nothing else
meshActor.GetProperty().SetRepresentationToWireframe()

### Hint
To remove actor from the renderer, use function renderer.RemoveActor(*actor_name*)

In [ ]:
# Set for interactive window (OpenGL)
"""
renwin = vtk.vtkRenderWindow()
renwin.AddRenderer(renderer)
renwin.SetSize(400, 300)

# An interactor
interactor = vtk.vtkRenderWindowInteractor()
# Add render to interactor window
interactor.SetRenderWindow(renwin)
# Start - Will display OpenGL interactive window with all actors
interactor.Initialize()
interactor.Start()
"""

In [ ]:
# Post-processing
# Color mesh by scalar value
# Add IPython element to embed window
# Function is taken from https://pyscience.wordpress.com/2014/09/03/ipython-notebook-vtk/
from IPython.display import Image
def vtk_show(renderer, scalarbar, width=400, height=300):
    """
    Takes vtkRenderer instance and returns an IPython Image with the rendering.
    """
    renderWindow = vtk.vtkRenderWindow()
    renderWindow.SetOffScreenRendering(1)
    renderWindow.AddRenderer(renderer)
    renderWindow.SetSize(width, height)
    renderWindow.Render()
     
    windowToImageFilter = vtk.vtkWindowToImageFilter()
    windowToImageFilter.SetInput(renderWindow)
    windowToImageFilter.Update()
    writer = vtk.vtkPNGWriter()
    writer.SetWriteToMemory(1)
    writer.SetInputConnection(windowToImageFilter.GetOutputPort())
    writer.Write()
    data = str(buffer(writer.GetResult()))
    
    return Image(data)

In [ ]:
# A renderer and render window
renderer = vtk.vtkRenderer()
# Define background color in RGB format
renderer.SetBackground(1, 1, 1)

In [ ]:
res_bar = vtk.vtkScalarBarActor()
res_bar.SetOrientationToVertical()
res_bar.SetNumberOfLabels(8)
res_bar.SetLabelFormat("%+#.1f")
res_bar.SetHeight(0.7)
res_bar.SetWidth(0.1)
res_bar.SetPosition(0.8,0.15)
res_bar.SetLookupTable(resActor.GetMapper().GetLookupTable())
# Set labels format
res_bar.GetLabelTextProperty().SetFontFamilyToCourier()
res_bar.GetLabelTextProperty().SetJustificationToRight()
res_bar.GetLabelTextProperty().SetVerticalJustificationToCentered()
res_bar.GetLabelTextProperty().BoldOff()
res_bar.GetLabelTextProperty().ItalicOff()
res_bar.GetLabelTextProperty().ShadowOff()        
res_bar.GetLabelTextProperty().SetColor(0, 0, 0)
# Set title format
res_bar.SetTitle("Ux, m/s")
res_bar.GetLabelTextProperty().SetFontFamilyToCourier()
res_bar.GetTitleTextProperty().SetColor(0,0,0)
res_bar.GetTitleTextProperty().BoldOff()

In [ ]:
# add the actors
# Plane actor
#enderer.AddActor(planeActor)
# Mesh actor
#renderer.AddActor(meshActor)
# Results actor
renderer.AddActor(resActor)
# Add scalarbar actor
renderer.AddActor(res_bar)
# Define render window (I put it there if we want to render in external OpenGL window)

In [ ]:
#renderer.RemoveActor(res_bar)
#renderer.AddActor(res_bar)

In [ ]:
# Show results in IPython window (non-interactively)
#renderer.RemoveActor(resActor)
# We can also define some camera settings
cam = renderer.GetActiveCamera()
#am.Azimuth(120.0)
#cam.Elevation(1.0)
#cam.Zoom(1.5)
#cam.Pitch(1.)
cam.SetParallelProjection(1)
cam.SetFocalPoint(0.05,0.05,0.0)
cam.SetPosition(0.05,0.05,0.01)
#cam.SetDistance(1.0)
vtk_show(renderer,res_bar,500,400)

In [ ]:
# render results using vtkOpenFOAMReader class
reader2 = vtkOpenFOAMReader()
reader2.SetFileName(os.path.join(os.getcwd(),"cavity/cavity.foam"))
reader2.Update()
reader2.SetTimeValue(0.5)
reader2_out = reader2.GetOutputPort(0)
reader2.SetCreateCellToPoint(1) # 0 if use Cell data, 1 if use Point data
#block0 = vtkUnstructuredGrid.SafeDownCast(reader2.GetOutput().GetBlock(0))

In [ ]:
polyFilter = vtkCompositeDataGeometryFilter()
polyFilter.SetInputConnection(0,reader2_out)

#print(polyFilter)
twoMapper = vtkPolyDataMapper()
twoMapper.SetInputConnection(polyFilter.GetOutputPort())
twoMapper.CreateDefaultLookupTable()
#twoMapper.SetScalarModeToUseCellFieldData()
twoMapper.SetScalarModeToUsePointFieldData() # use point data for coloring
twoMapper.SelectColorArray("U")
twoMapper.SetScalarRange(-0.2,1.5)
actor = vtk.vtkActor()
actor.SetMapper(twoMapper)

In [ ]:
# Other instruments

# Define cut plane (XZ plane)
plane = vtk.vtkPlane()
# Origin point
plane.SetOrigin(0.05,0.099,0.005)
# Normal
plane.SetNormal(0,0,1)

#create cutter
cutter=vtk.vtkCutter()
cutter.SetCutFunction(plane)
cutter.SetInputData(block0)
cutter.Update()
cutterMapper=vtk.vtkPolyDataMapper()
cutterMapper.SetInputConnection(cutter.GetOutputPort())

# Define plane actor
planeActor = vtk.vtkActor()
# The color of the plane
planeActor.GetProperty().SetColor(1.0,1.0,1)
planeActor.GetProperty().SetLineWidth(2)
planeActor.SetMapper(cutterMapper)
#planeActor.GetProperty().SetRepresentationToWireframe()

In [ ]:
rend = vtkRenderer()
rend.SetBackground(0.8,0.8,0.8)
rend.RemoveActor(actor)
rend.AddActor(planeActor)
vtk_show(rend,500,400)

In [ ]:
# renderer
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
ren.AddActor(planeActor)
ren.SetBackground(0.5, 0.5, 0.5) # Background color
renWin.SetSize(1024, 768)
# An interactor
interactor = vtk.vtkRenderWindowInteractor()
# Add render to interactor window
style = vtk.vtkInteractorStyleTrackballCamera()
interactor.SetInteractorStyle(style)
interactor.SetRenderWindow(renWin)
# Start - Will display OpenGL interactive window with all actors
interactor.Initialize()
interactor.Start()

In [ ]:
ColorRange = vtk.vtkLookupTable()
ColorRange.SetTableRange(0, 1)
ColorRange.SetHueRange(0, 1)
ColorRange.SetSaturationRange(1, 1)
ColorRange.SetAlphaRange(0.3, 0.5)
ColorRange.Build()
print(reader2.GetOutput())

In [ ]:
# Plot streamlines
source = vtk.vtkPlaneSource()
source.SetOrigin(0.05,0.05,0.005)
#source.SetNormal(0.0,1.0,0.0)
sphere = vtk.vtkSphereSource()
sphere.SetRadius(0.0001)
sphere.SetCenter(0.05, 0.05, 0.005) # Critical point for all 3 test datasets
sphere.SetThetaResolution(10)
integrator = vtk.vtkRungeKutta4()

stream = vtk.vtkStreamLine()
stream.SetInputConnection(pointData.GetOutputPort())

#stream.SetSourceConnection(sphere.GetOutputPort())
stream.SetStartPosition(0.05,0.099,0.005)
stream.SetIntegrator(integrator)
stream.SetMaximumPropagationTime(500)
stream.SetIntegrationStepLength(0.0001)
stream.SetIntegrationDirectionToIntegrateBothDirections()
#stream.SetStepLength(0.0001)

In [ ]:
scalarSurface = vtk.vtkRuledSurfaceFilter()
scalarSurface.SetInputConnection(stream.GetOutputPort())
scalarSurface.SetOffset(0)
scalarSurface.SetOnRatio(2)
scalarSurface.PassLinesOn()
scalarSurface.SetRuledModeToPointWalk()
scalarSurface.SetDistanceFactor(50)

tube = vtk.vtkTubeFilter()
tube.SetInputConnection(stream.GetOutputPort())
tube.SetRadius(0.0001)
#tube.SetNumberOfSides(6)

In [ ]:
"""
stream_mapper = vtk.vtkPolyDataMapper()
stream_mapper.SetInputConnection(stream.GetOutputPort())
streamActor = vtk.vtkActor()
streamActor.SetMapper(stream_mapper)
streamActor.VisibilityOn()
"""
dataMapper = vtk.vtkDataSetMapper()
dataMapper.SetInputConnection(tube.GetOutputPort())
dataMapper.SetScalarRange(-0.5, 1.0)
#dataMapper.SetLookupTable(ColorRange)

dataActor = vtk.vtkActor()
dataActor.SetMapper(dataMapper)

In [ ]:
# A renderer and render window
renderer2 = vtk.vtkRenderer()
# Define background color in RGB format
renderer2.SetBackground(1, 1, 1)

In [ ]:
renderer2.AddActor(planeActor)
renderer2.AddActor(meshActor)
#renderer2.RemoveActor(meshActor)

In [ ]:
cam2 = renderer2.GetActiveCamera()
#cam2.Azimuth(10.0)
#cam2.Elevation(1.0)
#cam2.Zoom(0.8)
#cam2.Pitch(-10.)
#cam2.SetParallelProjection(0)
cam2.SetFocalPoint(0.05,0.05,0.0)
#cam2.SetPosition(0.05,0.05,0.08)
#cam2.SetDistance(1.0)
#vtk_show(renderer2,res_bar,500,400)

In [ ]:

renwin = vtk.vtkRenderWindow()
renwin.AddRenderer(rend)
renwin.SetSize(600, 400)

# An interactor
interactor = vtk.vtkRenderWindowInteractor()
# Add render to interactor window
style = vtk.vtkInteractorStyleTrackballCamera()
interactor.SetInteractorStyle(style)
interactor.SetRenderWindow(renwin)
# Start - Will display OpenGL interactive window with all actors
interactor.Initialize()
interactor.Start()


In [ ]:
interactor.ResetCamera()